In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from pathlib import Path
from collections import Counter
import json
import nltk

nltk.download("punkt")
nltk.download("stopwords")

tf_idf = TfidfVectorizer(stop_words="english")


def encode_topics(df):
    # topics = df["topics"].str.get_dummies(sep=",")
    # topics = df["topics"].apply( topicfor topic in topics  )
    one_hot_encoded = (
        pd.get_dummies(df["topics"].apply(pd.Series).stack()).groupby(level=0).sum()
    )
    df = pd.concat([df, one_hot_encoded], axis=1)
    # print(df)
    return df


def set_index(df, index_column="poll_ID"):
    df.set_index(index_column, inplace=True)
    return df


def reset_index(df):
    df.reset_index()
    return df


def check_column_type(df, column_name, check_type):
    column_index = df.columns.get_loc(column_name)
    for i in range(len(df)):
        if not isinstance(df.iloc[i, column_index], check_type):
            print(
                f"error: {df.iloc[i, 0], df.iloc[i, 1],df.iloc[i, 2], df.iloc[i, 3], df.iloc[i, 4]}"
            )


def preprocess_text(text):
    tokens = nltk.tokenize.word_tokenize(text)
    # tokens = [word.lower() for word in tokens if type(word) is str]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in string.punctuation]
    stop_words = set(nltk.corpus.stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    processed_text = " ".join(tokens)

    return processed_text


def preprocess_list(field_list):
    ret_list = []
    stop_words = set(nltk.corpus.stopwords.words("english"))
    for item in field_list:
        tokens = nltk.tokenize.word_tokenize(item)
        # tokens = [word.lower() for word in tokens if type(word) is str]
        tokens = [word.lower() for word in tokens]
        tokens = [word for word in tokens if word not in string.punctuation]
        tokens = [word for word in tokens if word not in stop_words]
        processed_text = " ".join(tokens)
        ret_list.append(processed_text)

    return ret_list


def create_tf_idf_matrix(df, column):
    # print(f"{df[column]} is {df[column].dtype} and {df[column].dtype is list} {list}: ")
    df[column] = df[column].apply(lambda x: " ".join(x))
    df[column] = df[column].apply(preprocess_text)

    return tf_idf.fit_transform(df[column])


def create_souped_tf_idf_matrix(df):
    df["topics"] = df["topics"].apply(preprocess_list)
    df["question"] = df["question"].apply(preprocess_text)

    # Create a new soup feature
    df["soup"] = df.apply(create_soup, axis=1)

    return tf_idf.fit_transform(df["soup"])


def create_soup(df):
    res = (
        df["question"]
        + " "
        + " ".join(df["options"])
        + " "
        + (4 * (" " + " ".join(df["topics"])))
    )
    # print(f"-----------------------------------\n* Processing: [{ }]")
    return res


def calc_cosine_similarity_matrix(tf_idf_matrix_1, tf_idf_matrix_2):
    return cosine_similarity(tf_idf_matrix_1, tf_idf_matrix_2)


def id_to_index(df, id):
    return df[df["id"] == id].index.values[0]


def title_from_idx(df, idx):
    return df[df.index == idx]


def gen_recommendations(
    index,
    df,
    cosine_similarity_matrix,
    number_of_recommendations,
):
    # index = idx_from_title(df, original_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(
        similarity_scores, key=lambda x: x[1], reverse=True
    )

    recommendations_indices = [
        t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    ]
    recommendations = list(df["title"].iloc[recommendations_indices])
    # print(recommendations)
    # print(similarity_scores_sorted, type(similarity_scores_sorted))
    # recommendations_indices = [
    #    t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # recommendations_scores = [
    #    t[1] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # return (df["title"].iloc[recommendations_indices], recommendations_scores)

    return recommendations


def gen_rec_from_list_of_polls(
    interacted_polls, polls, cosine_similarity_matrix, number_of_recommendations
):
    recommendations = []
    for poll_id in interacted_polls:
        index = id_to_index(polls, poll_id)
        similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
        similarity_scores_sorted = sorted(
            similarity_scores, key=lambda x: x[1], reverse=True
        )

        recommendations_indices = [
            t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
        ]
        recs = list(polls["id"].iloc[recommendations_indices])

        # Filter out polls that have already been interacted with
        filtered_recs = [poll for poll in recs if poll not in interacted_polls]

        recommendations.append(filtered_recs)

    flattened_recommendations = [
        item for sublist in recommendations for item in sublist
    ]
    flattened_recommendations = Counter(flattened_recommendations)
    n_most_recommended = flattened_recommendations.most_common(
        number_of_recommendations
    )
    n_most_recommended = [t[0] for t in n_most_recommended]
    # print(n_most_recommended)

    return n_most_recommended

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>


In [2]:
from elasticsearch import Elasticsearch
import json


class ElasticsearchHandel:
    def __init__(self, elasticsearch_url, username, password, fingerprint):
        self.elasticsearch_url = elasticsearch_url
        self.username = username
        self.password = password
        self.fingerprint = fingerprint
        self.client = Elasticsearch(
            hosts=self.elasticsearch_url,
            basic_auth=(self.username, self.password),
            ssl_assert_fingerprint=self.fingerprint,
        )

    def get_index(self, index_name, batch_size=100):
        setattr(self, index_name, [])
        index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        while True:
            # query = {"query": {"match_all": {}}, "size": batch_size, "from": from_index}
            results = self.client.search(
                index=index_name,
                query={"match_all": {}},
                size=batch_size,
                from_=from_index,
            )
            instances = results["hits"]["hits"]

            all_instances.extend(instances)
            from_index += batch_size
            if len(instances) < 100:
                break

        setattr(self, index_name, [instance["_source"] for instance in all_instances])
        return getattr(self, index_name)

    def get_interactions(self, index_name, user_id, batch_size=100):
        # setattr(self, index_name, [])
        # index_list = getattr(self, index_name)
        from_index = 0
        all_instances = []

        query = {
            "match_phrase": {"userId": user_id},
        }

        results = self.client.search(
            index=index_name,
            query=query,
            size=batch_size,
            from_=from_index,
            timeout="1s",
        )
        # instances = results["hits"]["hits"][0]
        hits = results["hits"].get("hits")

        if not hits:
            # raise ValueError("User doesn't have any interactions.")
            raise InteractionNotFound()

        return hits[0].get("_source")

    def get_trend_polls(self, polls):
        # polls = getattr(self, "polls")
        # trend_polls = sorted(polls, key=lambda x: (-x["numberOfPollups"], -x["numberOfVotes"], -x["numberOfLike"]))
        trend_polls = sorted(
            polls,
            key=lambda x: (
                -x["numberOfVotes"],
                -x["numberOfLike"],
                # -x["numberOfPollUp"],
            ),
        )

        # recs = trend_polls["id"]

        # print("\n", filtered_trend_polls, "\n")
        # setattr(self, "trend_polls", trend_polls)
        return trend_polls

    def export_index_to_file(self, index, index_file_path):
        try:
            with open(index_file_path, "w") as output:
                # for instance in self.instances:
                #        json.dump(instance["_source"], output, indent=4)
                json.dump(index, output, indent=4)
        except Exception as exp:
            print("Export Error", exp)

In [3]:
import pandas as pd

elasticsearch_url = "https://159.203.183.251:9200"
username = "pollett"
password = "9r0&rJP@19GY"
fingerprint = "CE:AA:F7:FF:04:C7:31:14:78:9C:62:D4:CE:98:F9:EF:56:DA:70:45:37:14:E3:F8:66:0A:25:ED:05:04:83:ec"


elastic_handle = ElasticsearchHandel(elasticsearch_url, username, password, fingerprint)

polls = elastic_handle.get_index("polls")
trend_polls = elastic_handle.get_trend_polls(polls)

polls_df = pd.DataFrame.from_records(polls)

polls_df

,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,pollLimitations,isDeleted
0,c076c4b5-e3ae-4ca5-8191-00de0628a326,Do you research pet-friendly attractions and a...,"[Always, Rarely, Sometimes]","[Pets, Travel]",Public,a55320db-8ada-4e1f-8cc1-f8d5b52bbc0c,2023-09-28T09:52:15.043355Z,0,0,0,0,0,NaN,NaN
1,c1a67395-ce58-4df5-993a-cf48293f4513,Should athletes be required to align with cert...,"[No, Yes]","[Politics, Sport]",Public,0c6ef01a-de23-4c22-b72e-9a3f15b5fde2,2023-09-28T09:52:15.051446Z,0,0,0,0,0,NaN,NaN
2,c220777e-4a39-4401-a8b9-fe648fa8a6fc,Do you believe politicians try to exert contro...,"[No, Yes]","[Politics, Sport]",Public,0c6ef01a-de23-4c22-b72e-9a3f15b5fde2,2023-09-28T09:52:15.051441Z,0,0,0,0,0,NaN,NaN
3,c220fe81-7b78-4c89-b87e-d8c80cb80c67,Which paint finish provides the best resistanc...,"[Eggshell, Matte, Semi-gloss]","[Home Décor, Science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043387Z,0,0,0,0,0,NaN,NaN
4,c391295a-2629-43a3-b554-8f8140ba44a4,Have you ever taken your pet on a road trip?,"[No, Yes, Planning to in the future]","[Pets, Travel]",Public,a55320db-8ada-4e1f-8cc1-f8d5b52bbc0c,2023-09-28T09:52:15.04336Z,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,400165d0-c4ab-4203-8d48-31467b1d7e59,a,"[1, 2]",[General],Public,71046671-fd21-419c-b6b7-9f9fe13d286d,0001-01-01T00:00:00,1,0,0,1,2,"{'allowedGender': 'Men', 'allowedAgeRange': {'...",False
2125,a7c3cca3-428c-4983-94bc-d16fa3dc59bb,What do you like to do in the future?,"[Electrician , Docter, Plumber]","[General, Activity]",Public,e00b366a-37a8-407d-9a15-e585d1ad539a,0001-01-01T00:00:00,0,0,1,0,2,NaN,False
2126,54767fb6-621c-4e00-b425-ddb0895d4ae7,Which state is the best?,"[NY, LA]",[General],Public,e00b366a-37a8-407d-9a15-e585d1ad539a,0001-01-01T00:00:00,1,0,1,0,1,NaN,False
2127,9f5d9de3-4543-4ddc-b419-6f139dd3080b,Which one for my birthday?,"[, ]","[General, Beauty, Fashion]",Public,e00b366a-37a8-407d-9a15-e585d1ad539a,0001-01-01T00:00:00,0,0,0,0,0,NaN,False


In [24]:
test_limitations = {
    "allowedLocations": [
        {"country": "United States", "city": "United States", "state": "NJ"},
        {"country": "United States", "city": "United States", "state": "NJ"},
        {"country": "United States", "city": "United States", "state": "NJ"},
    ],
    "allowedGender": "All",
    "allowedAgeRange": {"minimumAge": 0, "maximumAge": 120},
    "isDeleted": False,
}

user_limitations = {
    "Location": {"country": "United States", "city": "United States", "state": "NJ"},
    "Gender": "Men",
    "Age": 30,
}

In [42]:
# Function to check if the values are in the expected format
def is_valid_limitations(limitations):
    return (
        "allowedLocations" in limitations
        and "allowedGender" in limitations
        and "allowedAgeRange" in limitations
    )


def filter_polls(row, user_limitations):
    if (
        row["pollType"] == "Public"
        and isinstance(row.get("pollLimitations"), dict)
        and all(k in user_limitations for k in ["Location", "Gender", "Age"])
    ):
        user_location = user_limitations.get("Location")

        allowed_locations = row.get("pollLimitations").get("allowedLocations")
        if len(allowed_locations) == 0 or any(
            user_location == loc for loc in allowed_locations
        ):
            allowed_gender = row["pollLimitations"]["allowedGender"]
            user_gender = user_limitations["Gender"]
            if allowed_gender == "All" or allowed_gender == user_gender:
                allowed_age_range = row["pollLimitations"]["allowedAgeRange"]
                user_age = user_limitations["Age"]
                if (
                    allowed_age_range["minimumAge"]
                    <= user_age
                    <= allowed_age_range["maximumAge"]
                ):
                    return True

    return False

In [43]:
# Filter the DataFrame based on the user's limitations
# filtered_df = polls_df[polls_df['pollLimitations'].apply(lambda x: meets_limitations(x, test_user))]
filtered_df = polls_df[polls_df.apply(filter_polls, args=(user_limitations,), axis=1)]

In [44]:
filtered_df

,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,pollLimitations,isDeleted,soup
157,1edc7b0b-df6b-439f-81ee-fc654f3ce4f6,wich one,"[Sketch, XD, Figma]","[tech, sport]",Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-05-07T06:24:48.218491Z,2,7,4,0,1,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,wich one Sketch XD Figma tech sport tech spor...
158,1ec1f108-2c7b-45d5-819c-9db42ff7a9ab,one,"[one, two]","[activity, tech]",Public,66271a97-73ba-41c8-b460-23d166e4c020,2023-04-27T09:06:50.953044Z,2,3,0,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,one one two activity tech activity tech activ...
160,3c269d38-d5c0-4e94-b101-a6d745a6bc66,,"[Taskoloo, Trello, Click up]","[sport, activity, sport, activity, activity]",Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-05-04T09:56:55.615412Z,2,4,4,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,Taskoloo Trello Click up sport activity spor...
161,4230b1ae-a544-4bea-9603-e1cb08954f5c,animal,"[Cat , Dog]",[],Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-04-30T08:05:21.159336Z,1,3,3,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,animal Cat Dog
162,51aa5d4a-3347-412f-939f-4711ed473729,one,"[2, 1]","[tech, sport, politics]",Public,66271a97-73ba-41c8-b460-23d166e4c020,2023-04-27T08:55:26.12225Z,0,0,0,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,one 2 1 tech sport politics tech sport politi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,9fcebbae-d565-43ed-870a-9c09b1cbcbb1,test,"[1, 2]",[general],Public,e00b366a-37a8-407d-9a15-e585d1ad539a,0001-01-01T00:00:00,0,0,0,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,test 1 2 general general general general
2121,1035b0ca-01d4-464f-9a26-8a2aad960230,fitness-related youtube channel follow,"[Popsugar Fitness, Other, Yoga with Adriene, B...","[tech, activity]",Public,57c29fe3-2e5f-4872-b338-0475aee4578c,2023-09-28T09:52:15.049541Z,0,0,1,2,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,fitness-related youtube channel follow Popsuga...
2122,0bdfe92f-7a04-4ba5-8efa-ba30fa9076ca,fitness-related mobile app find useful,"[Other, Couch to 5K, Headspace, Strava, Fitbod]","[activity, tech]",Public,57c29fe3-2e5f-4872-b338-0475aee4578c,2023-09-28T09:52:15.049516Z,0,1,0,0,1,"{'allowedLocations': [], 'allowedGender': 'All...",False,fitness-related mobile app find useful Other C...
2123,aa64c7e1-4e1d-4efb-ad84-af47a9527d5a,one,"[1, 2]",[general],Public,67eb27ca-ba0b-4d29-8627-9ec78327b512,0001-01-01T00:00:00,0,0,0,0,1,"{'allowedGender': 'Men', 'allowedAgeRange': {'...",False,one 1 2 general general general general


In [46]:
import requests

# API URL
url = "https://dev.pollett.io/api/Recommend/Polls/GetPrivatePollThatUserCanSee"

# Parameters
params = {"userId": "bbe64b34-ba34-4fbd-a62f-e6c84c0423b4"}

# Send a GET request to the API
response = requests.get(url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json().get("data")
    # Process the data as needed
    print(data)
else:
    # Handle the error
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

allowed_private_polls = polls_df[polls_df["id"].isin(data)]


allowed_private_polls

['e749a6a3-592b-42be-a041-2eb155a9e95c', '9378ff77-4f78-4c7d-8ea7-4dc5c1e363f2']


,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,pollLimitations,isDeleted,soup
1907,e749a6a3-592b-42be-a041-2eb155a9e95c,social media platoform prefer conduct poll,"[Facebook, Instagram , LinkedIn, Pollett , Twi...","[tech, politics, science]",Private,08f0071c-397c-420d-a1fb-32f613a73398,2023-05-14T15:44:22.843829Z,4,12,8,1,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,social media platoform prefer conduct poll Fac...
2128,9378ff77-4f78-4c7d-8ea7-4dc5c1e363f2,one,"[1, 2]",[general],Private,edf0904d-d63a-42c4-842e-a6cd460168fe,0001-01-01T00:00:00,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,one 1 2 general general general general


In [47]:
nan_limitation_count = polls_df["pollLimitations"].isna().sum()
len(polls_df)
nan_limitation_count

159

In [51]:
concatenated_df = pd.concat([filtered_df, allowed_private_polls], axis=0)

# Reset the index if needed
concatenated_df.reset_index(drop=True, inplace=True)

concatenated_df

,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,pollLimitations,isDeleted,soup
0,1edc7b0b-df6b-439f-81ee-fc654f3ce4f6,wich one,"[Sketch, XD, Figma]","[tech, sport]",Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-05-07T06:24:48.218491Z,2,7,4,0,1,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,wich one Sketch XD Figma tech sport tech spor...
1,1ec1f108-2c7b-45d5-819c-9db42ff7a9ab,one,"[one, two]","[activity, tech]",Public,66271a97-73ba-41c8-b460-23d166e4c020,2023-04-27T09:06:50.953044Z,2,3,0,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,one one two activity tech activity tech activ...
2,3c269d38-d5c0-4e94-b101-a6d745a6bc66,,"[Taskoloo, Trello, Click up]","[sport, activity, sport, activity, activity]",Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-05-04T09:56:55.615412Z,2,4,4,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,Taskoloo Trello Click up sport activity spor...
3,4230b1ae-a544-4bea-9603-e1cb08954f5c,animal,"[Cat , Dog]",[],Public,4c208829-0b25-47a6-8574-b10fc3db7ab2,2023-04-30T08:05:21.159336Z,1,3,3,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,animal Cat Dog
4,51aa5d4a-3347-412f-939f-4711ed473729,one,"[2, 1]","[tech, sport, politics]",Public,66271a97-73ba-41c8-b460-23d166e4c020,2023-04-27T08:55:26.12225Z,0,0,0,0,0,"{'allowedGender': 'All', 'allowedAgeRange': {'...",False,one 2 1 tech sport politics tech sport politi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,0bdfe92f-7a04-4ba5-8efa-ba30fa9076ca,fitness-related mobile app find useful,"[Other, Couch to 5K, Headspace, Strava, Fitbod]","[activity, tech]",Public,57c29fe3-2e5f-4872-b338-0475aee4578c,2023-09-28T09:52:15.049516Z,0,1,0,0,1,"{'allowedLocations': [], 'allowedGender': 'All...",False,fitness-related mobile app find useful Other C...
1947,aa64c7e1-4e1d-4efb-ad84-af47a9527d5a,one,"[1, 2]",[general],Public,67eb27ca-ba0b-4d29-8627-9ec78327b512,0001-01-01T00:00:00,0,0,0,0,1,"{'allowedGender': 'Men', 'allowedAgeRange': {'...",False,one 1 2 general general general general
1948,400165d0-c4ab-4203-8d48-31467b1d7e59,,"[1, 2]",[general],Public,71046671-fd21-419c-b6b7-9f9fe13d286d,0001-01-01T00:00:00,1,0,0,1,2,"{'allowedGender': 'Men', 'allowedAgeRange': {'...",False,1 2 general general general general
1949,e749a6a3-592b-42be-a041-2eb155a9e95c,social media platoform prefer conduct poll,"[Facebook, Instagram , LinkedIn, Pollett , Twi...","[tech, politics, science]",Private,08f0071c-397c-420d-a1fb-32f613a73398,2023-05-14T15:44:22.843829Z,4,12,8,1,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,social media platoform prefer conduct poll Fac...


In [52]:
polls_tf_idf_matrix = create_souped_tf_idf_matrix(concatenated_df)
# print(polls_tf_idf_matrix[0,1])
# print(polls_tf_idf_matrix[0].data)
# print(polls_tf_idf_matrix[0])

polls_tf_idf_matrix

<1951x5379 sparse matrix of type '<class 'numpy.float64'>'
	with 26285 stored elements in Compressed Sparse Row format>

The `polls_tf_idf_matrix` is a sparse matrix used to represent textual data in a numerical format. Let's break down its characteristics:

- **Dimensions**: The matrix has dimensions of 142 rows and 274 columns.

- **Sparse Matrix**: It's classified as a sparse matrix, meaning that the majority of its elements are zero. This is common in text data like TF-IDF matrices, where most terms do not appear in every document.

- **Data Type**: The elements of the matrix are of type `numpy.float64`, representing 64-bit floating-point numbers. This is the standard data type for TF-IDF values.

- **Stored Elements**: There are 586 non-zero elements (entries) in the matrix. Sparse matrices are memory-efficient because they only store these non-zero values.

- **Compressed Sparse Row Format (CSR)**: The matrix is stored in the Compressed Sparse Row (CSR) format, a widely used format for sparse matrices. It allows for efficient row-wise access and arithmetic operations.

In summary, the `polls_tf_idf_matrix` efficiently represents TF-IDF values of text data with 142 rows and 274 columns. Its sparse nature optimizes memory usage by storing only non-zero values, making it suitable for text analysis tasks.


In [11]:
cosine_similarity_matrix = calc_cosine_similarity_matrix(
    polls_tf_idf_matrix, polls_tf_idf_matrix
)
cosine_similarity_matrix.shape

(2129, 2129)

**Cosine Similarity Matrix Explanation:**

- **Definition:** The `cosine_similarity_matrix` is a matrix designed to represent the similarity between pairs of polls within a dataset.

- **Calculation:** It is calculated using the `calc_cosine_similarity_matrix` function, which commonly utilizes the cosine similarity metric. Cosine similarity is a frequently used measure in natural language processing and information retrieval. It assesses the similarity between two vectors, in this context, the TF-IDF vectors representing the polls.

- **Interpretation:** In the `cosine_similarity_matrix`, each element `(i, j)` denotes the cosine similarity between two polls: poll `i` and poll `j`. The values within this matrix have a range from -1 to 1, and their meanings are as follows:

    - `1`: Indicates that the polls are identical or have the highest possible similarity.
    - `0`: Denotes that the polls are orthogonal, implying no similarity between them.
    - `-1`: Suggests that the polls are diametrically opposite or possess the highest possible dissimilarity.

This matrix is critical for generating recommendations as it quantifies the textual content's similarity or dissimilarity between different polls. By leveraging this similarity matrix, the recommendation system can identify polls with content similar to those the user has interacted with, resulting in more personalized and relevant recommendations.


In [12]:
# user_id = request.args.get("userId")
user_id = "67eb27ca-ba0b-4d29-8627-9ec78327b512"


userInteractions = elastic_handle.get_interactions("userpollinteractions", user_id)

userInteractions = [
    interaction["pollId"] for interaction in userInteractions["userPollActions"][:20]
]
recommended_list = gen_rec_from_list_of_polls(
    userInteractions,
    polls_df,
    cosine_similarity_matrix,
    100,
)

recommended_polls = polls_df[polls_df["id"].isin(recommended_list)]
recommended_polls

,id,question,options,topics,pollType,ownerId,createdAt,numberOfLike,numberOfViews,numberOfVotes,numberOfComments,numberOfPollUp,pollLimitations,isDeleted,soup
3,c220fe81-7b78-4c89-b87e-d8c80cb80c67,paint finish provides best resistance moisture...,"[Eggshell, Matte, Semi-gloss]","[home décor, science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043387Z,0,0,0,0,0,NaN,NaN,paint finish provides best resistance moisture...
8,c7b3c378-04b8-4db2-b579-b9625fd0cf4f,type insulation material environmentally frien...,"[Hemp, Recycled denim, Sheep's wool]","[home décor, science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043417Z,0,0,0,0,0,NaN,NaN,type insulation material environmentally frien...
13,db2a2fe6-2476-4ed2-9310-e1ebcdf69f8b,type lighting energy-efficient home decor,"[CFL bulbs, Halogen bulbs, LED bulbs]","[home décor, science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043369Z,0,0,0,0,0,NaN,NaN,type lighting energy-efficient home decor CFL ...
28,a7881fbe-983d-4bd8-a866-034223e08bba,ideal angle solar panels maximize energy produ...,"[50-60 degrees, 10-20 degrees, 30-40 degrees]","[science, home décor]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043406Z,0,0,0,0,0,NaN,NaN,ideal angle solar panels maximize energy produ...
29,a7990b9b-cc36-4484-b15b-826d26fbfac8,color scheme promotes productivity home office...,"[Yellow and orange, Blue and green, Red and bl...","[home décor, science]",Public,72edb639-4231-44d4-92a2-ac7310bd6745,2023-09-28T09:52:15.043372Z,0,0,0,0,0,NaN,NaN,color scheme promotes productivity home office...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,f5151226-191d-4c7b-af26-1480355a7b7e,color scheme prefer living room,"[Neutral tones, Bright and bold colors, Subtle...",[home décor],Public,a2a14c2b-eacf-405b-8f79-ace81728ba3b,2023-09-28T09:52:15.046291Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,color scheme prefer living room Neutral tones ...
2029,f82409a6-b543-407d-80ff-055279055ce7,ideal humidity level storing books prevent dam...,"[40-50%, 60-70%, 30-40%]","[home décor, science]",Public,0724901b-5f89-44ff-a249-29c51a8e6985,2023-09-28T09:52:15.043444Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,ideal humidity level storing books prevent dam...
2034,f944a2ac-55d7-463d-999e-b11850ce8d4a,type window glass provides best insulation hea...,"[Single glazed, Double glazed, Triple glazed]","[home décor, science]",Public,0724901b-5f89-44ff-a249-29c51a8e6985,2023-09-28T09:52:15.043508Z,0,2,1,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,type window glass provides best insulation hea...
2059,fb6b0f5d-1b57-471d-859f-1f3f7402c061,type flooring material lowest carbon footprint...,"[Cork, Bamboo, Linoleum]","[science, home décor]",Public,3ae14a1d-d340-45d1-a796-f66d901c35aa,2023-09-28T09:52:15.043543Z,0,0,0,0,0,"{'allowedLocations': [], 'allowedGender': 'All...",False,type flooring material lowest carbon footprint...
